In [ ]:
import xarray as xr
from gcsfs import mapping
import pickle
from dask_kubernetes import KubeCluster as Cluster
from dask.distributed import Client

%matplotlib inline

In [ ]:
cluster = Cluster(n_workers=10)
client = Client(cluster)
cluster

In [ ]:
def get_ws_p(pix_deg, da_masks, da_p, tolerance=None):
    if tolerance is None:
        s = str(pix_deg)
        tolerance = 10 ** (-(len(s) - s.find('.')))
    p = (da_p.reindex_like(da_masks, method='nearest', tolerance=tolerance).clip(0) * da_masks)
    return p

In [ ]:
gpm_zarr = mapping.GCSMap('pangeo-data/gpm_imerg_early')
ds_p = xr.open_zarr(gpm_zarr)
ds_p

In [ ]:
das = {}
das['maroni'] = xr.open_zarr('../data/maroni_mask')
das['oyapock'] = xr.open_zarr('../data/oyapock_mask')
da = xr.concat(list(das.values()), 'label')
da_masks = da.assign_coords(label=list(das))
da_masks = da_masks.where(da_masks==1.)

In [ ]:
from_time, to_time = '2019-03-01', '2019-04-01'
da_p = ds_p.precipitationCal.sel(time=slice(from_time, to_time))
p = get_ws_p(0.1, da_masks, da_p).chunk({'time': -1, 'label': 1})
p

In [ ]:
p = p.mask.compute()
p.sel(label='maroni', time='2019-03-01T00:15:00').plot.imshow()

In [ ]:
df = p.to_dataframe()
df.to_csv('precipitation.csv')